# CIS 5200: Machine Learning
## Homework 3

In [ ]:
import os
import sys

# For autograder only, do not modify this cell.
# True for Google Colab, False for autograder
NOTEBOOK = (os.getenv('IS_AUTOGRADER') is None)
if NOTEBOOK:
    print("[INFO, OK] Google Colab.")
else:
    print("[INFO, OK] Autograder.")
    sys.exit()

### Penngrader setup

In [ ]:
# %%capture
!pip install penngrader-client

In [ ]:
%%writefile config.yaml
grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'

In [ ]:
from penngrader.grader import PennGrader

# PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO
# TO ASSIGN POINTS TO YOU IN OUR BACKEND
STUDENT_ID = 00000000 # YOUR PENN-ID GOES HERE AS AN INTEGER #
SECRET = STUDENT_ID

grader = PennGrader('config.yaml', 'cis5200_sp24_HW3', STUDENT_ID, SECRET)

In [ ]:
# packages for homework
import torch
import torch.nn.functional as F
import torch.nn as nn

from sklearn import datasets
import pandas as pd

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

### 0. Gradients with PyTorch

At this point, you've implemented a lot of gradients. However, these days, manual implementation of gradients is a thing of the past: PyTorch is a scientific computing library that comes with the ability to automatically compute gradients for you! This is called auto-differentiation. Here is an example of using auto-differentiation to compute the gradient of a quadratic function, $f(x) = ax^2$. The key parts are as follows:

1. Variables that you want to differentiate with respect to should have the `requires_grad` flag set to `True`.
2. Calculate the objective that you'd like the compute the gradient of, using the variable from step (1).
3. Pass the objective and the variable you are differentiating to `torch.autograd.grad`.

In [ ]:
# Step 1: Set requires_grad to true for x
x = torch.Tensor([3.0])
a = torch.Tensor([1.5])
x.requires_grad = True

# Step 2: Compute the objective
y = a*(x**2)

# Step 3: Use autograd
grad = torch.autograd.grad([y],[x])[0]

print("PyTorch gradient:", grad)
print("Analytic gradient:", 2*a*x)

You'll notice that the gradient computed with PyTorch matches exactly the analytic gradient $\nabla f(x) = 2ax$, but without having to implement or derive the analytic gradient! This works for gradients with respect to any sized variables. For example, if $x$ is now a vector, and the objective is $f(x) = a\|x\|_2^2$ then we can calculate the gradient in the same way:

In [ ]:
# Step 1: Set requires_grad to true for x
x = torch.Tensor([3.0, 2.0])
a = torch.Tensor([1.5])
x.requires_grad = True

# Step 2: Compute the objective
y = a*(x.norm(p=2)**2)

# Step 3: Use autograd
grad = torch.autograd.grad([y],[x])[0]

print("PyTorch gradient:", grad)
print("Analytic gradient:", 2*a*x)

From now on, we highly recommend that you use auto-differentiation to calculate gradients. As long as all of your operations are differentiable, the final objective will be differentiable.


# 1. SVM and Gradient Descent

In this first problem, you'll implement (soft margin) support vector machines with gradient descent, using gradients from PyTorch's autodifferentiation library.
+ (2pts) Calculate the objective of the Soft SVM
+ (2pts) Calculate the gradient of the Soft SVM objective
+ (2pts) Implement a basic gradient descent optimizer. Your solution needs to converge to an accurate enough answer.
+ (1pts) Make predictions with the Soft SVM

Tips:
- This assignment is more freeform than previous ones. You're allowed to initialize the parameters of the SVM model however you want, as long as your implemented functions return the right values.
- We recommend using PyTorch's `torch.autograd.grad` to get the gradient instead of deriving the SVM gradient.
- You'll need to play with the values of step size and number of iterations to
converge to a good value.
- To debug your optimization, print the objective over iterations. Remember that the theory says as long as the learning rate is small enough, for strongly convex problems, we are guaranteed to converge at a certain rate. What does this imply about your solution if it is not converging?
- As a sanity check, you can get around 97.5% prediction accuracy and converge to an objective below 0.16.  

In [ ]:
class SoftSVM():
    def __init__(self, ndims):
        # Here, we initialize the parameters of your soft-SVM model for binary
        # classification. You can change the initialization but don't change
        # the weight and bias variables as the autograder will assume that
        # these exist.
        # ndims := integer -- number of dimensions
        # no return type

        self.weight = torch.randn(ndims)
        self.bias = torch.randn(1)
        self.weight.requires_grad = True
        self.bias.requires_grad = True

    def objective(self, X, y, l2_reg):
        # Calculate the objective of your soft-SVM model
        # X := Tensor of size (m,d) -- the input features of m examples with d dimensions
        # y := Tensor of size (m) -- the labels for each example in X
        # l2_reg := float -- L2 regularization penalty
        # Returns a scalar tensor (zero dimensional tensor) -- the loss for the model
        # Fill in the rest
        pass
        return torch.zeros(1)


    def gradient(self, X, y, l2_reg):
        # Calculate the gradient of your soft-SVM model
        # X := Tensor of size (m,d) -- the input features of m examples with d dimensions
        # y := Tensor of size (m) -- the labels for each example in X
        # l2_reg := float -- L2 regularization penalty
        # Return Tuple (Tensor, Tensor) -- the tensors corresponds to the
        # gradients of the weight and bias parameters respectively
        # Fill in the rest
        pass
        return torch.zeros_like(self.weight), torch.zeros_like(self.bias)

    def optimize(self, X, y, l2_reg):
        # Calculate the gradient of your soft-SVM model
        # X := Tensor of size (m,d) -- the input features of m examples with d dimensions
        # y := Tensor of size (m) -- the labels for each example in X
        # l2_reg := float -- L2 regularization penalty

        # no return type

        # Fill in the rest
        pass

    def predict(self, X):
        # Given an X, make a prediction with the SVM
        # X := Tensor of size (m,d) -- features of m examples with d dimensions
        # Return a tensor of size (m) -- the prediction labels on the dataset X

        # Fill in the rest
        pass
        return torch.zeros(X.size(0))

Test the Soft SVM on the breast cancer dataset:

In [ ]:
#Load dataset
cancer = datasets.load_breast_cancer()
X,y = torch.from_numpy(cancer['data']), torch.from_numpy(cancer['target'])
mu,sigma = X.mean(0,keepdim=True), X.std(0,keepdim=True)
X,y = ((X-mu)/sigma).float(),(y - 0.5).sign() # prepare data
l2_reg = 0.1
print(X.size(), y.size())

# Optimize the soft-SVM with gradient descent
clf = SoftSVM(X.size(1))
clf.optimize(X,y,l2_reg)
print("\nSoft SVM objective: ")
print(clf.objective(X,y,l2_reg).item())
print("\nSoft SVM accuracy: ")
(clf.predict(X) == y).float().mean().item()

In [ ]:
grader.grade(test_case_id = 'SVM_objective', answer = SoftSVM)
grader.grade(test_case_id = 'SVM_gradient', answer = SoftSVM)
grader.grade(test_case_id = 'SVM_optimize', answer = SoftSVM)
grader.grade(test_case_id = 'SVM_predict', answer = SoftSVM)

### 2. Extending Reverse Mode Auto-differentiation (12 points)

In lecture we learned about how auto-differentiation could be used to automatically calculate gradients through a computational graph. Crucially, all we needed to do was know how to propagate variations of the chain rule at each individual node in the graph.

The PyTorch framework makes it very simple to extend auto-differentation to new formula or expressions. Specifically, PyTorch keeps track of the computational graph with the `forward` function, saving relevant computations, and then computes the chain rule with the `backward` function (in reverse mode).

For example, consider the Legendre polynomial of degree 3, $P_3(x) = \frac{1}{2}(5x^3 - 3x)$. How would we implement our own custom module to do this, if it didn't already exist in PyTorch? We can do this like in the following ([example taken from the PyTorch documetnation](https://pytorch.org/docs/stable/notes/extending.html)):  

In [ ]:
class LegendrePolynomial3(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        ctx.save_for_backward(input)
        return 0.5 * (5 * input ** 3 - 3 * input)

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        input, = ctx.saved_tensors
        return grad_output * 1.5 * (5 * input ** 2 - 1)

And that's it! We can now use the `LegendrePolynomial3` function in combination with any other PyTorch functions and automatically calculate gradients with auto-differentiation. Note that, since PyTorch uses *reverse-mode*, in the `backward` function we are:  
1. Given the gradient of the loss with respect to the output of the function, and
2. Need to recompute the gradient of the loss with respect to the inputs of the function.

This is an extremely general and powerful framework. For example, researchers have implemented forward functions and their gradients for modules as complex as a call to an external simulator such as a physics engine, which would then let you differentiate through the simulator!

In this exercise, you'll implement a new PyTorch function for a simpler function, the integral:
$$f(x) = \int_{-\infty}^x g(t)dt$$
PyTorch doesn't have a function for doing numerical integration, so we'll need to implement the backwards function ourselves if we want to use auto-differentiate with integrals. In particular, we'll do this for the following piece-wise constant function:

$$g(x; \beta, \eta) = \sum_{r=1}^R \beta_r \mathbb 1[x\in (\eta_{r-1},\eta_r)]$$

for parameters $\beta\in \mathbb R^R$ and $\eta\in \mathbb R^{R+1}$.

1. Forward (2pts): Implement the `forward` function which calculates $f(x)$ for a batch of example $x$.
2. Backward (6pts): Implement the `backward` function which calculates the reverse-mode auto-differentiation rule for $f$ with respect to $x,\beta, \eta$, 2 points each.

Hints:
+ Recall that $g(x)$ is a piece-wise constant function, and the integral of a function is the area between the function and 0. Thus, you can reformulate the integral of each constant part of $g$ as simply the (signed) area of the rectangle, and so the integral of $g(x)$ is the sum of the areas of all the piece-wise rectangles up to $x$
+ Recall that for reverse-mode autodifferentiation, we are given the derivative of the loss with respect to the output, or $\frac{\partial \ell(x))}{\partial f(x; \beta, \eta)}$, and our goal is to then compute $\frac{\partial \ell(x))}{\partial x}, \frac{\partial \ell(x))}{\partial \beta}, \frac{\partial \ell(x))}{\partial \eta}$.
+ Remember the chain rule: $$\frac{\partial \ell(x))}{\partial x} = \sum_i \frac{\partial \ell(x))}{\partial f(x; \beta, \eta)_i} \cdot \frac{\partial f(x; \beta, \eta))_i}{\partial x}$$
+ For simplicity, you can assume that all $x$'s and $\eta$'s are distinct so you don't have to worry about border cases for the intervals.

In [ ]:
class IntegralPiecewiseConstant(torch.autograd.Function):
    @staticmethod
    def forward(ctx, X, beta, eta):
        # ctx := A PyTorch object for saving data for use in the backward
        #   function
        # X := Tensor(float) of size (m) -- a minibatch of examples
        # beta := Tensor(float) of size (k) -- the magnitudes of each part of
        #   the piece-wise constant function
        # eta := Tensor(float) of size (k+1) -- the start/end points of each
        #   part of the piece-wise constant function
        # Return the integral of the piece-wise constant function applied to
        #   each entry of  X

        # Here we save the inputs to the forward function to use in backward
        ctx.save_for_backward(X, beta, eta)

        # Fill in the rest
        pass
        return torch.zeros(X.size(0))

    @staticmethod
    def backward(ctx, grad_output):
        # ctx := A PyTorch object for containing saved data from the forward
        #   function
        # grad_output := Tensor(float) of size (m) -- a minibatch of gradients
        #   of the loss with respect to the output of this function. Since we
        #   are working with a scalar function, each element is the gradient
        #   with respect to an output of the minibatch. In other words, this is
        #   dL/df(x) where this module outputs f(x).
        # Return a tuple containing the gradient of the loss with respect to
        #   the inputs X, beta, and eta: (dL/dX, dL/dbeta, dL/deta).

        # Here we retrieve the tensors from the forward pass
        X, beta, eta = ctx.saved_tensors
        print(grad_output)

        # Fill in the rest
        pass
        return torch.zeros_like(X), torch.zeros_like(beta), torch.zeros_like(eta)

As an example, calculating the gradients for the following example will result in the following output:
```
tensor([ 1., -1.,  3.])
tensor([0.2500, 0.4000, 0.7000, 0.1000])
tensor([-3., -2.,  6., -4.,  0.])
```

In [ ]:
X = torch.Tensor([0.05,0.5,0.9])
betas = torch.Tensor([1,2,-1,3])
etas = torch.Tensor([0,0.1,0.3,0.8,1])

X.requires_grad = True
betas.requires_grad = True
etas.requires_grad = True
IntegralPiecewiseConstant.apply(X, betas, etas).sum().backward()

print(X.grad)
print(betas.grad)
print(etas.grad)

In [ ]:
grader.grade(test_case_id = 'autodiff_forward', answer = IntegralPiecewiseConstant.forward)
grader.grade(test_case_id = 'autodiff_backward', answer = IntegralPiecewiseConstant.backward)

### 3. Neural Networks and Gradient Descent

In the previous example, we directly calculated the gradient of a function with respect to various inputs using PyTorch's `autograd` library. As we did in that problem, one can use this autograd library to directly implement gradient descent by iterating over all parameters and applying the gradient update. However, as the number of parameters grow, directly implementing these updates can become quite onerous. To handle neural networks with lots of parameters, the PyTorch library includes optimizers that make training with gradient descent very easy with the following 5 steps:
1. Create an optimizer object and give it all the parameters you'd like to optimize
2. Calculate a loss that you'd like to minimize
3. Clear old gradients
4. Calculate new gradients
5. Update the parameters with one gradient step

The end result is a generic boilerplate recipe that will optimize *any* objective with gradient descent. Here is an example running gradient descent on a linear model, using the stochastic gradient descent (SGD) optimizer and a basic dataloader.

In [ ]:
# Setup a simple problem
m,d = 128,5
X = torch.randn(m,d)
w_opt, b_opt = torch.randn(d), torch.randn(1)
y = X.matmul(w_opt) + b_opt

# setup the dataloader
simple_dataset = torch.utils.data.TensorDataset(X,y)
loader = torch.utils.data.DataLoader(simple_dataset,batch_size=16)

# Create the model
lin = nn.Linear(d,1)

# setup the optimizer (Step 1)
opt = torch.optim.SGD(lin.parameters(), lr=0.001)

# iterate over epochs
for i in range(100):
    # iterate over minibatches
    for X0,y0 in loader:
        yhat = lin(X0).squeeze(1) # make predictions
        loss = F.mse_loss(yhat,y0) # calculate loss (Step 2)

        opt.zero_grad() # clear gradients from previous iteration (Step 3)
        loss.backward() # calculate new gradients (Step 4)
        opt.step() # update parameters (Step 5)

    # logging
    with torch.no_grad():
        if i % 10 == 0:
            print(loss.item())



In the second part of this assignment, we'll implement a basic neural network for a tree cover classification problem using the PyTorch library. Here, the problem is to use 12 features (which have been expanded to 54 columns of data to expand categorical variables into binary features) to predict one of 7 tree cover types. A full description of the dataset can be found [here](http://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.info). The following cell will download the data and convert it into PyTorch tensors for you, with a training split of 2000 examples per class and a validation split of 500 examples per class.

In [ ]:
!pip install ucimlrepo

from ucimlrepo import fetch_ucirepo

# fetch dataset
covertype = fetch_ucirepo(id=31)

# data (as pandas dataframes)
X = covertype.data.features
y = covertype.data.targets

# metadata
# print(covertype.metadata)

# variable information
# print(covertype.variables)

In [ ]:
df = pd.concat([covertype.data.features, covertype.data.targets], axis = 1)
df = pd.concat([df.iloc[df.values[:,-1]==i].sample(2500) for i in range(1,8)])
X,y = df.values[:,:-1], df.values[:,-1]-1 # re-index labels to 0-6
X,y = [torch.from_numpy(a) for a in (X,y)] # convert to PyTorch

dataset = torch.utils.data.TensorDataset(X,y)
train_set,val_set = torch.utils.data.random_split(dataset,[2000*7,500*7]) # generate splits

Your goal is to achieve at least 70% accuracy on this forest cover task. We suggest a very simple neural network.

1. (5pts) Implement a model for predicting forest cover. You will get 1 point for every first 14% accuracy, up to 70% for a total of 5 points. This is achievable with a small neural network with one hidden layer and ReLU activations. It is possible to achieve over 80% accuracy with less than 15 seconds (many solutions exist that take less than a minute).

It is possible to get much higher than 75%, in fact it is possible to exceed 80% accuracy. **You do not need a GPU**.

Hints:
+ A simple neural network directly on the data can get around 50% accuracy. You can start with a sequential model, linear layers, and ReLU activations (PyTorch has a wide range of modules [here](https://pytorch.org/docs/stable/nn.html)).
+ To iterate over minibatches, PyTorch has a useful `DataLoader` object with documentation [here](https://pytorch.org/docs/stable/data.html?highlight=dataloader#torch.utils.data.DataLoader).
+ Data normalization can be a very important factor, not just in deep learning. Check your dataset statistics and normalize to improve your accuracy further.
+ You may need to explore different types of optimizers and learning rates.  
+ Remember to check your performance on the validation set, and not the training set. You will need to perform well on the held-out test data.
+ To help with your testing, you can add additional parameters via keyword arguments to tweak your pipeline.
+ There is no single right answer here---the only goal is to get to 70%!
+ Solutions that exploit the grader to extract test set labels will receive no credit.

In [ ]:
class ForestCover():
    def __init__(self, train_set):
        # train_set := a PyTorch dataset of training examples from the tree
        #   cover prediction problem.

        # Initialize your model here!
        pass

    def train(self, train_set):
        # train_set := a PyTorch dataset of training examples from the tree
        #   cover prediction problem.

        # Train your model here!
        pass

    def predict(self, X):
        # X := a Tensor(float) of size (m,d) of examples for the model to
        #   to predict.

        # Make predictions on a new input here!
        return torch.randn(X.size(0))

solution = ForestCover(train_set)
solution.train(train_set)

acc = []
for X,y in torch.utils.data.DataLoader(val_set, batch_size=128, shuffle=False):
    y_pred = solution.predict(X)
    acc.append(y_pred == y)

acc = torch.cat(acc).float().mean()
print(f"Validation accuracy: {acc:.2f}")

Your code will be scored according to accuracy on a held-out test set.
If you don't use your validation set during training, your performance on the validation set will be approximately your performance on the test set.
**Warning: solutions that exploit the grader to extract test set labels will receive a manual adjustment for zero credit.**

In [ ]:
!wget https://machine-learning-upenn.github.io/assets/hw3/X_test.pth -O "X_test.pth"
X_test = torch.load("X_test.pth")

In [ ]:
y_soln = solution.predict(X_test)
grader.grade(test_case_id = 'forestcover', answer = y_soln)